In [0]:
!pip install bs4
!pip install six
!pip install Arrays
!pip install gensim
!pip install requests

In [0]:
!pip install google-cloud-vision

    100% |████████████████████████████████| 389kB 26.3MB/s 


In [0]:
!pip install google-cloud-language

In [0]:
from google.colab import drive
drive.mount('/gdrive')

Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).


In [0]:
!cp ../../gdrive/My\ Drive/GoogleNews-vectors-negative300.bin.gz ./

In [0]:
import os
import io
import sys
from bs4 import BeautifulSoup
import requests
import array as arr
import six as six
import gensim

from google.cloud import vision
from google.cloud import language
from google.cloud.language import enums
from google.cloud.language import types

credential_path = "key.json"
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = credential_path

model1 = gensim.models.KeyedVectors.load_word2vec_format('GoogleNews-vectors-negative300.bin.gz', binary=True, limit = 500000)
credible = ['economictimes.', 'huffingtonpost.', 'theprint.', 'thelogicalindian.', 'thequint.', 'altnews.', 'wsj.', 'nypost.', 'nytimes.', 'bbc.', 'reuters.', 'economist.', 'pbs.', 'aljazeera.', 'thewire.', 'theatlantic.', 'theguardian.', 'edition.cnn',
            'cnbc.', 'scroll.in', 'financialexpress.', 'npr.', 'usatoday.', 'snopes.', 'politifact.']

In [0]:
def entity_sentiment_text(text):
    """Detects entity sentiment in the provided text."""
    client = language.LanguageServiceClient()

    if isinstance(text, six.binary_type):
        text = text.decode('utf-8')

    document = types.Document(
        content=text.encode('utf-8'),
        type=enums.Document.Type.PLAIN_TEXT)

    # Detect and send native Python encoding to receive correct word offsets.
    encoding = enums.EncodingType.UTF32
    if sys.maxunicode == 65535:
        encoding = enums.EncodingType.UTF16

    result = client.analyze_entity_sentiment(document, encoding)

    #for entity in result.entities:
        #print('Mentions: ')
        #print(u'Name: "{}"'.format(entity.name))
        #for mention in entity.mentions:
            #print(u'  Begin Offset : {}'.format(mention.text.begin_offset))
            #print(u'  Content : {}'.format(mention.text.content))
            #print(u'  Magnitude : {}'.format(mention.sentiment.magnitude))
            #print(u'  Sentiment : {}'.format(mention.sentiment.score))
            #print(u'  Type : {}'.format(mention.type))
        #print(u'Salience: {}'.format(entity.salience))
        #print(u'Sentiment: {}\n'.format(entity.sentiment))

#---------------------#--------------------#---------------------#--------------------#
#Function for google's clous vision API
def detect_web(path):
    list = []
    i = 0
    """Detects web annotations given an image."""
    from google.cloud import vision
    client = vision.ImageAnnotatorClient()

    with io.open(path, 'rb') as image_file:
        content = image_file.read()

    image = vision.types.Image(content=content)

    response = client.web_detection(image=image)
    annotations = response.web_detection

    # if annotations.best_guess_labels:
    #     for label in annotations.best_guess_labels:
    #         print('\nBest guess for the image: {}'.format(label.label))
    #         print("--------------------------------------------------------------------------------")


    if annotations.pages_with_matching_images:
        # print('\n{} Pages with matching images found:'.format(
        #     len(annotations.pages_with_matching_images)))

        for page in annotations.pages_with_matching_images:
            # print('\n\tPage url   : {}'.format(page.url))
            list.append(page.url)

    # if annotations.web_entities:
    #     print('\n{} Web entities found in the image: '.format(
    #         len(annotations.web_entities)))

    #     for entity in annotations.web_entities:
    #         print('\n\tScore      : {}'.format(entity.score))
    #         print(u'\tDescription: {}'.format(entity.description))

    # if annotations.visually_similar_images:
    #     print('\n{} visually similar images found:\n'.format(
    #         len(annotations.visually_similar_images)))

    #     for image in annotations.visually_similar_images:
    #         print('\tImage url    : {}'.format(image.url))
    # print("--------------------------------------------------------------------------------")
    return(list)
#---------------------#--------------------#---------------------#--------------------#
#Function to check which URLs belong to credible news sources
def credible_list(list_of_page_urls):

    c_list = []

    c_length = len(credible)

    url_length = len(list_of_page_urls)

    f = [[0 for j in range(c_length)] for i in range(url_length)]
    for i in range(url_length):
        for j in range(c_length):
            f[i][j] = list_of_page_urls[i].find(credible[j])
            if((list_of_page_urls[i].find(credible[j])) > 0):
                c_list.append(list_of_page_urls[i])
    if c_list == []:
        print("No credible sources have used this image, please perform human verification.")
        print("--------------------------------------------------------------------------------")
        exit(1)
    return(c_list)
#---------------------#--------------------#---------------------#--------------------#
#Function to scrape titles off the given URLs
def titles(credible_from_url_list):
    title_list = []
    for urls in credible_from_url_list:
        if urls != []:
            r = requests.get(urls)
            html = r.content
            soup = BeautifulSoup(html, 'html.parser')
            title_list.append(soup.title.string)
    return(title_list)

#---------------------#--------------------#---------------------#--------------------#
# #Function to print the scraped titles
# def print_article_title(title_list):
#     print("Credible article titles which use the same image: ")
#     print("--------------------------------------------------------------------------------")
#     for title in title_list:
#         print(title)
#         print("--------------------------------------------------------------------------------")
#---------------------#--------------------#---------------------#--------------------#
#Function to call google's language API for entity analysis
def entity_analysis(title_list):
    for title in title_list:
        entity_sentiment_text(title)

#---------------------#--------------------#---------------------#--------------------#
#Function to compute the WM distances between titles and associated title and the average distance
def wmdist(given_title,title_list):
    # print("Word Mover's Distance for Titles:")
    # print("--------------------------------------------------------------------------------")
    distances = []
    for title in title_list:
        dist = model1.wmdistance(given_title, title) #determining WM distance
        distances.append(dist)
        #distance = model1.WmdSimilarity(given_title, title)

    sum_dist = 0
    for distance in distances:
        sum_dist = sum_dist + distance
        # print ('distance = %.3f' % distance)
        # print("--------------------------------------------------------------------------------")

    avg_dist = sum_dist/len(distances)
    # print("Average Distance: {}".format(avg_dist))
    # print("--------------------------------------------------------------------------------")
    return(avg_dist)

#---------------------#--------------------#---------------------#--------------------#
#Function to decide whether human verification is required
def human_ver(avg_dist):
    if(avg_dist >= 1.0):
        return 0
    else:
        return 1

In [0]:
def check(given_title,image_path):
    list_of_page_urls = []
    credible_from_url_list = []
    title_list = []
    list_of_page_urls = detect_web(image_path)
    credible_from_url_list = credible_list(list_of_page_urls)
    title_list = titles(credible_from_url_list)
    entity_analysis(title_list)
    avg_dist = wmdist(given_title, title_list)
    score = human_ver(avg_dist)
    return list_of_page_urls, credible_from_url_list, title_list

In [0]:
list_of_page_urls, credible_from_url_list, title_list = check('PM Modi Wave a Pakistani Flag','modi.jpeg')

In [0]:
list_of_page_urls

['https://www.thequint.com/news/webqoof/fake-doctored-image-narendra-modi-waving-green-flag-pakistan',
 'http://newsmobile.in/articles/2019/02/22/pm-modi-didnt-wave-pakistani-flag-dont-believe-this-photoshopped-picture/',
 'https://timesofindia.indiatimes.com/news/fact-check-photo-showing-pm-modi-with-islamic-flag-is-fake/articleshow/68106706.cms',
 'https://www.indiatoday.in/fact-check/story/fact-check-morphed-image-shows-pm-modi-waving-islamic-flag-1461707-2019-02-21',
 'https://www.vishvasnews.com/english/politics/fact-check-pm-did-not-wave-pakistani-flag-the-viral-image-is-photoshopped/',
 'https://www.boomlive.in/doctored-image-of-pm-modi-goes-viral/',
 'https://www.msn.com/es-xl/noticias/fotos/sex-racket-busted-in-meerut/vp-BBKuIgw',
 'https://facthunt.in/rumour/73/PM-Modi-waved-Islamic-Flag:-No,-the-image-is-photoshopped',
 'https://www.msn.com/en-in/news/other/adopt-a-heritage-dalmia-group-adopts-red-fort-tajmahal/vp-AAwsiaD',
 'https://www.hotstar.com/news/fact-check-video-did

In [0]:
credible_from_url_list

['https://www.thequint.com/news/webqoof/fake-doctored-image-narendra-modi-waving-green-flag-pakistan']

In [0]:
title_list

['Fact Check: No, PM Modi Didn’t Wave a Pakistani Flag, it’s Photoshopped']

In [0]:
score

1



---



In [0]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import seaborn as sns
%matplotlib inline

np.random.seed(2)

from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
import itertools

from keras.utils.np_utils import to_categorical # convert to one-hot-encoding
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D
from keras.optimizers import RMSprop
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ReduceLROnPlateau, EarlyStopping

sns.set(style='white', context='notebook', palette='deep')

from PIL import Image
import os
from pylab import *
import re
from PIL import Image, ImageChops, ImageEnhance

In [0]:
def get_imlist(path):
    return [os.path.join(path,f) for f in os.listdir(path) if f.endswith('.jpg') or f.endswith('.png')]

def convert_to_ela_image(path, quality):
    filename = path
    resaved_filename = filename.split('.')[0] + '.resaved.jpg'
    ELA_filename = filename.split('.')[0] + '.ela.png'
    
    im = Image.open(filename).convert('RGB')
    im.save(resaved_filename, 'JPEG', quality=quality)
    resaved_im = Image.open(resaved_filename)
    
    ela_im = ImageChops.difference(im, resaved_im)
    
    extrema = ela_im.getextrema()
    max_diff = max([ex[1] for ex in extrema])
    if max_diff == 0:
        max_diff = 1
    scale = 255.0 / max_diff
    
    ela_im = ImageEnhance.Brightness(ela_im).enhance(scale)
    
    return ela_im

In [0]:
model = Sequential()

model.add(Conv2D(filters = 32, kernel_size = (5,5),padding = 'valid', 
                 activation ='relu', input_shape = (128,128,3)))
print("Input: ", model.input_shape)
print("Output: ", model.output_shape)

model.add(Conv2D(filters = 32, kernel_size = (5,5),padding = 'valid', 
                 activation ='relu'))
print("Input: ", model.input_shape)
print("Output: ", model.output_shape)

model.add(MaxPool2D(pool_size=(2,2)))

model.add(Dropout(0.25))
print("Input: ", model.input_shape)
print("Output: ", model.output_shape)

model.add(Flatten())
model.add(Dense(256, activation = "relu"))
model.add(Dropout(0.5))
model.add(Dense(2, activation = "softmax"))

optimizer = RMSprop(lr=0.0005, rho=0.9, epsilon=1e-08, decay=0.0)
model.compile(optimizer = optimizer , loss = "categorical_crossentropy", metrics=["accuracy"])

Input:  (None, 128, 128, 3)
Output:  (None, 124, 124, 32)
Input:  (None, 128, 128, 3)
Output:  (None, 120, 120, 32)
Input:  (None, 128, 128, 3)
Output:  (None, 60, 60, 32)


In [0]:
model.load_weights('../../gdrive/My Drive/model_image-05-0.9427.hdf5')

In [0]:
x_test_arr = []
x_test = 'modi.jpeg'
x_test_arr.append(array(convert_to_ela_image(x_test, 50).resize((128, 128))).flatten() / 255.0)
x_test_arr = np.array(x_test_arr)
x_test_arr = x_test_arr.reshape(-1, 128, 128, 3)
model.predict(x_test_arr)

array([[0.71227366, 0.28772634]], dtype=float32)

In [0]:
def just_image_check(image_path):
  # 0 - Fake, 1- Real
  x_test_arr = []
  x_test = image_path
  x_test_arr.append(array(convert_to_ela_image(x_test, 50).resize((128, 128))).flatten() / 255.0)
  x_test_arr = np.array(x_test_arr)
  x_test_arr = x_test_arr.reshape(-1, 128, 128, 3)
  arr = model.predict_classes(x_test_arr)
  return arr[0]

In [0]:
just_image_check('./modi.jpeg')

0